In [ ]:
import torch.nn as nn
import torch
import MST
import numpy as np

In [ ]:
np.random.seed(42)

In [ ]:
in_C = 1
out_C = 1
stride = 1
padding = 1
dilation = 1
kernel = 3

params = [in_C, out_C, kernel, stride, padding, dilation]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")

In [ ]:
with torch.no_grad():
    myConv = MST.Conv2d(*params)
    torchConv = nn.Conv2d(*params)

    torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
    torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())


In [ ]:
image = np.ones((1, in_C, 16, 16))

In [ ]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[:20])
    print(nnres[:20])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

In [ ]:
%%timeit
myres = myConv(image)

In [ ]:
%%timeit
nnres = torchConv(torch.Tensor(image))

In [2]:
def to_np_arr(a):
    a_ = MST.MDT_ARRAY(a).astype(np.float32) / 255
    return a_.reshape(1, *a_.shape)

import torch.nn as nn
import torch
import MST
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
np.random.seed(42)
in_C = 1
out_C = 6
stride = 1
padding = 0
dilation = 1
kernel = 5
inH, InW = 32, 32
outH, outW = inH//stride, InW//stride

params = [in_C, out_C, kernel, stride, padding, dilation]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")
myConv = MST.Conv2d(*params)
torchConv = nn.Conv2d(*params)

torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())
image = np.ones((1, in_C, inH, InW))

transform = to_np_arr

12.0


In [3]:
# torchImage = torch.ones((1, in_C, inH, InW), requires_grad=True)

from tqdm.auto import tqdm
train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
)

for (images, labels) in tqdm(train_dataloader):

    torchImage = images
    image = torchImage.detach().numpy()
    
    out_sample = myConv(image)

    dOut = np.random.random(out_sample.shape)

    myres = myConv(image)
    myres.backward(dOut)
    nnres = torchConv(torchImage)
    nnres.backward(torch.Tensor(dOut))

    # nngrad = torchImage.grad.numpy().round(3).flatten()
    # mygrad = myConv._dinX.round(3).flatten()
    # bads = np.where(abs(mygrad - nngrad) > 0.001)
    # print(len(bads[0]))
    # for pos in bads[0][:10]:
    #     print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    nnres = nnres.detach().numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

    nngrad = torchConv.weight.grad.numpy().round(3).flatten()
    mygrad = myConv._dw.round(3).flatten()
    bads = np.where(abs(mygrad - nngrad) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    torchConv.weight.grad = torch.zeros_like(torchConv.weight.grad)

    nngrad = torchConv.bias.grad.numpy().round(3).flatten()
    mygrad = myConv._dbias.round(3).flatten()
    bads = np.where(abs(mygrad - nngrad) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    torchConv.bias.grad = torch.zeros_like(torchConv.bias.grad)

  0%|          | 0/1875 [00:00<?, ?it/s]

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [3]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[:20])
    print(nnres[:20])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

IndexError: index 30 is out of bounds for axis 2 with size 30

In [15]:
import torch.nn as nn
import torch
import MST
import numpy as np
np.random.seed(42)
BS = 513
in_C = 1
out_C = 6
stride = 4
padding = 2
dilation = 1
kernel = 5
inH, InW = 256, 256
outH, outW = inH//stride, InW//stride

params = [kernel, stride, padding, dilation]

# print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")
myConv = MST.MaxPool2d(*params)
torchConv = nn.MaxPool2d(*params)

# torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
# torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())
# image = np.ones((BS, in_C, inH, InW))
torchImage = torch.rand((BS, in_C, inH, InW), requires_grad=True)
image = torchImage.detach().numpy()

In [16]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[54:70])
    print(nnres[54:70])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

(513, 1, 64, 64)
torch.Size([513, 1, 64, 64])
[0.935 0.945 0.986 0.976 0.89  0.944 0.909 0.951 0.964 0.95  0.913 0.989
 0.927 0.968 0.992 0.944]
[0.935 0.945 0.986 0.976 0.89  0.944 0.909 0.951 0.964 0.95  0.913 0.989
 0.927 0.968 0.992 0.944]
0


In [17]:


out_sample = myConv(image)

myConv(image).backward(np.ones(out_sample.shape))
torchConv(torchImage).backward(torch.ones(out_sample.shape))


nngrad = torchImage.grad.numpy().round(3).flatten()
mygrad = myConv._dinX.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")


nngrad = torchConv.weight.grad.numpy().round(3).flatten()
mygrad = myConv._dw.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

torchConv.weight.grad = torch.zeros_like(torchConv.weight.grad)

nngrad = torchConv.bias.grad.numpy().round(3).flatten()
mygrad = myConv._dbias.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

torchConv.bias.grad = torch.zeros_like(torchConv.bias.grad)

0


AttributeError: 'MaxPool2d' object has no attribute 'weight'